In [1]:
import requests
from bs4 import BeautifulSoup as bts
import pandas as pd
import re
import numpy as np
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
def getAndParseURL_selenium(url):
    result = driver.get(url)
    soup = bts(driver.page_source, 'html5lib')
    return soup

In [3]:
def getAndParseURL(url):
    result = requests.get(url)
    soup = bts(result.text, 'html.parser')
    return soup

In [4]:
#kişisel gelişim sekmesindeki kitaplara ulaşma
driver=webdriver.Chrome()
driver.get("https://www.kitapyurdu.com/")
driver.maximize_window()
button=driver.find_element(By.XPATH,'//*[@id="mainNav"]/div[1]/ul/li[1]/div[2]/ul/li[3]/span')
button.click()
driver.implicitly_wait(1)
button_2=driver.find_element(By.XPATH,'//*[@id="mainNav"]/div[1]/ul/li[1]/div[2]/ul/li[3]/div/div[1]/div/ul[2]/li[19]/a')
button_2.click()

In [5]:
pages=["https://www.kitapyurdu.com/kategori/kitap-kisisel-gelisim/341.html"]
for page in range (2,149):
    pages.append("https://www.kitapyurdu.com/index.php?route=product/category&page=" + str(page)+"&path=341&filter_in_stock=1")
pages

['https://www.kitapyurdu.com/kategori/kitap-kisisel-gelisim/341.html',
 'https://www.kitapyurdu.com/index.php?route=product/category&page=2&path=341&filter_in_stock=1',
 'https://www.kitapyurdu.com/index.php?route=product/category&page=3&path=341&filter_in_stock=1',
 'https://www.kitapyurdu.com/index.php?route=product/category&page=4&path=341&filter_in_stock=1',
 'https://www.kitapyurdu.com/index.php?route=product/category&page=5&path=341&filter_in_stock=1',
 'https://www.kitapyurdu.com/index.php?route=product/category&page=6&path=341&filter_in_stock=1',
 'https://www.kitapyurdu.com/index.php?route=product/category&page=7&path=341&filter_in_stock=1',
 'https://www.kitapyurdu.com/index.php?route=product/category&page=8&path=341&filter_in_stock=1',
 'https://www.kitapyurdu.com/index.php?route=product/category&page=9&path=341&filter_in_stock=1',
 'https://www.kitapyurdu.com/index.php?route=product/category&page=10&path=341&filter_in_stock=1',
 'https://www.kitapyurdu.com/index.php?route=p

In [6]:
links=[]
for page in pages:
    html=getAndParseURL_selenium(page)
    for sonuc in html.findAll('a' , {"class" : "pr-img-link"}):
        links.append(sonuc.get('href'))

In [7]:
len(links)

2960

In [8]:
result = []
for link in links:
    
    try:
        browser=webdriver.Chrome()   
        browser.get(link)
        button_3 = browser.find_element(By.CSS_SELECTOR,"#rating-" + str(link).split("/")[-1].rstrip("html&path=341").rstrip('.')+"> ul")
        button_3.click()
        element = WebDriverWait(browser, 10).until( EC.presence_of_element_located((By.CSS_SELECTOR, '.rating-ratio')) ) 
        rating_ratio = float(element.text.split()[0]) 
    except:
        rating_ratio=np.nan
    
    html = getAndParseURL(link)
    
    try:
        price=float(html.find("div",{"class":"price__item"}).text.strip().replace(",","."))
    except:
        price = np.nan  
    try:
        rating_value=int(html.find("meta", {"itemprop":"ratingValue"}).get("content").replace(".",""))
    except:
        rating_value=np.nan
    try: 
        review_count = int(html.find("meta", {"itemprop":"reviewCount"}).get("content").replace(".",""))
    except:
        review_count=np.nan  
    try:
        cilt_tipi=html.find("div",{"class":"attributes"}).find(text=re.compile("Cilt Tipi")).findNext().text.strip()
    except:
        cilt_tipi=np.nan
    try:
        kagit_cinsi=html.find("div",{"class":"attributes"}).find(text=re.compile("Kağıt Cinsi")).findNext().text.strip()
    except:
        kagit_cinsi=np.nan
    try:
        book_name=html.find("h1" , {"class" : "pr_header__heading"}).text
    except:
        book_name=np.nan
    try:
        will_read_count=int(html.find("span" , {"class" : "readlist-count" , "id" : "count-1"}).text.split()[0].replace(".",""))
    except:
        will_read_count=np.nan
    try:
        now_reading_count=int(html.find("span" , {"class" : "readlist-count" , "id" : "count-2"}).text.split()[0])
    except:
        now_reading_count=np.nan
    try:
        already_read_count=int(html.find("span" , {"class" : "readlist-count" , "id" : "count-3"}).text.split()[0].replace(".",""))
    except:
        already_read_count=np.nan
    try:
        favourite_count=int(html.find("span" , { "id" : "favorite-count"}).text.split()[0].replace(".",""))
    except:
        favourite_count=np.nan
    try:
        purchase_info=int(html.find("p",{"class":"purchased"}).text.split()[2].replace(".",""))
    except:
        purchase_info=np.nan
    try:
        page_num=html.find("div",{"class":"attributes"}).find(text=re.compile("Sayfa Sayısı:")).findNext().text.strip()
    except:
        page_num=np.nan
    
    result.append([price,rating_value,review_count,cilt_tipi,kagit_cinsi,rating_ratio,book_name,will_read_count,now_reading_count,already_read_count,favourite_count,purchase_info,page_num])
    browser.quit()



In [9]:
result

[[94.62,
  5,
  268,
  'Karton Kapak',
  'Kitap Kağıdı',
  nan,
  'Atomik Alışkanlıklar',
  1616,
  271,
  869,
  21580,
  39005,
  '352'],
 [12.9,
  5,
  121,
  'Karton Kapak',
  'Kitap Kağıdı',
  92.8,
  'İrade Terbiyesi',
  487,
  70,
  374,
  5404,
  15536,
  '232'],
 [78.12,
  4,
  6,
  'Karton Kapak',
  'Kitap Kağıdı',
  89.1,
  'O An',
  75,
  13,
  34,
  686,
  2590,
  '248'],
 [74.87,
  5,
  15,
  'Karton Kapak',
  'Kitap Kağıdı',
  97.0,
  'Yaşadım Demek İçin Ne Yapmalı?',
  138,
  15,
  37,
  2415,
  2756,
  '232'],
 [110.05,
  4,
  238,
  'Karton Kapak',
  'Kitap Kağıdı',
  89.4,
  'Beden Dili  Eski FBI Ajanından İnsanların Bedenini Okuma Rehberi',
  479,
  66,
  418,
  7608,
  7845,
  '300'],
 [79.3,
  5,
  28,
  'Karton Kapak',
  'Kitap Kağıdı',
  94.7,
  'Sınırlar  Hayatınızı Kontrol Etmek İçin Ne Zaman Evet Nasıl Hayır Demelisiniz',
  104,
  26,
  63,
  2083,
  4789,
  '312'],
 [237.05,
  5,
  29,
  'Karton Kapak',
  'Kitap Kağıdı',
  98.6,
  'Hayatın Kaynağı',
  154,
 

In [10]:
columns = ["price","rating_value","review_count","cover_type","paper_type","rating_ratio","book_name","will_read_count","now_reading_count","already_read_count","favourite_count","purchase_info","page_num"]
df = pd.DataFrame.from_records(result,columns=columns)

In [11]:
df

,price,rating_value,review_count,cover_type,paper_type,rating_ratio,book_name,will_read_count,now_reading_count,already_read_count,favourite_count,purchase_info,page_num
0,94.62,5.0,268.0,Karton Kapak,Kitap Kağıdı,NaN,Atomik Alışkanlıklar,1616.0,271.0,869.0,21580.0,39005.0,352
1,12.90,5.0,121.0,Karton Kapak,Kitap Kağıdı,92.8,İrade Terbiyesi,487.0,70.0,374.0,5404.0,15536.0,232
2,78.12,4.0,6.0,Karton Kapak,Kitap Kağıdı,89.1,O An,75.0,13.0,34.0,686.0,2590.0,248
3,74.87,5.0,15.0,Karton Kapak,Kitap Kağıdı,97.0,Yaşadım Demek İçin Ne Yapmalı?,138.0,15.0,37.0,2415.0,2756.0,232
4,110.05,4.0,238.0,Karton Kapak,Kitap Kağıdı,89.4,Beden Dili Eski FBI Ajanından İnsanların Bede...,479.0,66.0,418.0,7608.0,7845.0,300
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2955,119.63,5.0,1.0,Karton Kapak,Kitap Kağıdı,90.0,Yeniden Matematik Aşısı,1.0,1.0,1.0,24.0,28.0,118
2956,56.00,4.0,3.0,Karton Kapak,Kitap Kağıdı,70.0,Arıza Bir Arıza'nın Anatomisi,2.0,0.0,4.0,31.0,63.0,190
2957,94.25,5.0,7.0,Karton Kapak,Kitap Kağıdı,94.7,Yakıt Deponuzu Dolu Tutun,9.0,2.0,6.0,76.0,156.0,184
2958,46.75,NaN,NaN,Karton Kapak,Kitap Kağıdı,0.0,Kimse Sana Karşı Değil,0.0,0.0,0.0,5.0,12.0,121


In [14]:
df.to_csv('kitap_yurdu.csv')
df.to_pickle('kitap_yurdu_survey_data.pkl')

In [13]:
df_2=pd.read_csv('kitap_yurdu.csv')
df_2

,Unnamed: 0,price,rating_value,review_count,cover_type,paper_type,rating_ratio,book_name,will_read_count,now_reading_count,already_read_count,favourite_count,purchase_info,page_num
0,0,94.62,5.0,268.0,Karton Kapak,Kitap Kağıdı,NaN,Atomik Alışkanlıklar,1616.0,271.0,869.0,21580.0,39005.0,352.0
1,1,12.90,5.0,121.0,Karton Kapak,Kitap Kağıdı,92.8,İrade Terbiyesi,487.0,70.0,374.0,5404.0,15536.0,232.0
2,2,78.12,4.0,6.0,Karton Kapak,Kitap Kağıdı,89.1,O An,75.0,13.0,34.0,686.0,2590.0,248.0
3,3,74.87,5.0,15.0,Karton Kapak,Kitap Kağıdı,97.0,Yaşadım Demek İçin Ne Yapmalı?,138.0,15.0,37.0,2415.0,2756.0,232.0
4,4,110.05,4.0,238.0,Karton Kapak,Kitap Kağıdı,89.4,Beden Dili Eski FBI Ajanından İnsanların Bede...,479.0,66.0,418.0,7608.0,7845.0,300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2955,2955,119.63,5.0,1.0,Karton Kapak,Kitap Kağıdı,90.0,Yeniden Matematik Aşısı,1.0,1.0,1.0,24.0,28.0,118.0
2956,2956,56.00,4.0,3.0,Karton Kapak,Kitap Kağıdı,70.0,Arıza Bir Arıza'nın Anatomisi,2.0,0.0,4.0,31.0,63.0,190.0
2957,2957,94.25,5.0,7.0,Karton Kapak,Kitap Kağıdı,94.7,Yakıt Deponuzu Dolu Tutun,9.0,2.0,6.0,76.0,156.0,184.0
2958,2958,46.75,NaN,NaN,Karton Kapak,Kitap Kağıdı,0.0,Kimse Sana Karşı Değil,0.0,0.0,0.0,5.0,12.0,121.0
